# Clean a JSON LinkedIn profile file
Takes a JSON file that was scrapped with data from LinkedIn and makes it ready to be used. Converts the _position_ titles, normalizes all the text cases. This should be used on a raw JSON scrapped file as first set before adapting it to Dropcontact format.

In [13]:
import re
import json
import pandas
import pathlib
import secrets
import datetime
from itertools import chain
from nltk.tokenize import LineTokenizer
from marketing_data_cleaning import DATA_FOLDER_PATH


In [14]:

def chain_for_dataframe(filename, write_file=True):
    """
    Regroups subsets of dictionnaries together

    >>> [[{...}], [{...}]]
    ... [{...}, {...}]
    """
    with open(DATA_FOLDER_PATH / filename, mode='r', encoding='utf-8') as f:
        chained_values = list(chain(*json.load(f)))
        df = pandas.DataFrame(chained_values)

        if write_file:
            df.to_json(
                DATA_FOLDER_PATH / 'chained_output.json',
                force_ascii=False,
                orient='records'
            )
        return df


df = chain_for_dataframe(DATA_FOLDER_PATH / 'inputs/v8.json')


In [15]:
df['company'].describe()


count                                     765
unique                                     31
top       (16) ASSADIA : personnes | LinkedIn
freq                                      155
Name: company, dtype: object

## Clean main data
Make cases title case (especially first_name and last_name)

In [16]:
tokenizer = LineTokenizer()

for item in df.itertuples():
    if item.company_description is not None:
        tokens = tokenizer.tokenize(item.company_description) 
        df.loc[item.Index, 'company_description'] = ' '.join(tokens)


In [17]:
def normalize_names(value):
    return str(value).lower().title()


columns_to_normalize = ['last_name', 'first_name', 'full_name']

for column in columns_to_normalize:
    df[column] = df[column].apply(normalize_names)


In [18]:
def extract_company(value):
    return_value = None
    result = re.match(r'^\(\d+\)\s(.*)\s?\:', str(value))
    if result:
        return_value = result.group(1).strip().upper()
    return return_value or value.upper()

df['company'] = df['company'].apply(extract_company)


In [19]:
# for item in df.itertuples():
#     regexes = [
#         r'chez\s?\w+',
#         rf'{item.company}'
#     ]

#     new_value = None
#     for regex in regexes:
#         result = re.search(regex, item.position)
#         if result is None:
#             continue
#         new_value = re.sub(regex, '', item.position)
#         df.loc[item.Index, 'position'] = new_value or item.position


In [20]:
df = df.drop_duplicates(subset=['first_name', 'last_name'])


In [21]:
df = df.sort_values('last_name')


# Airtable adapter
Adapts a Dropcontact adapated CSV file by cleaning and correcting the column names so that it can eventually be uploaded to an Airtable base.

In [22]:
COLUMNS_TO_ADAPT = {
    'linkedin': 'LinkedIn',
    'courtesy_title': 'Civilité',
    'first_name': 'Prénom',
    'last_name': 'Nom',
    'full_name': 'Nom complet',
    'position': 'Poste',
    'company': 'Entreprise',
    'company_linkedin': 'Company LinkedIn',
    'enriched': 'Statut enrichissement',
    'email': 'Email',
    'website': 'Site entreprise',
    'company_metadata': 'Company metadata',
    'company_members': 'Company members',
    'company_description': 'Description'
}
airtable_df = df.rename(columns=COLUMNS_TO_ADAPT)


In [23]:
interesting_positions = [
    'directeur', 'directrice', 'director', 'ceo', 'dirigeant', 'ceo', 'founder',
    'responsable', 'manager', 'founder', 'fondateur', 'fondatrice', 
    'développeur', 'développement', 'development', 'gérant', 'business', 
    'entrepreneur', 'franchisé'
]


def test_position(value):
    result = False

    if value is None or value == '' or value == '-':
        return result

    for position in interesting_positions:
        if position in value.lower():
            result = True
            break
    return result


airtable_df['of_interest'] = airtable_df['Poste'].map(test_position)


In [24]:
# 'Civilité',
airtable_df = airtable_df[['Prénom', 'Nom', 'Nom complet', 'Poste', 'LinkedIn', 'Entreprise', 'Company LinkedIn',
                           'Statut enrichissement', 'Site entreprise', 'Company metadata', 'Company members', 'Description']]


airtable_df.columns


Index(['Prénom', 'Nom', 'Nom complet', 'Poste', 'LinkedIn', 'Entreprise',
       'Company LinkedIn', 'Statut enrichissement', 'Site entreprise',
       'Company metadata', 'Company members', 'Description'],
      dtype='object')

In [32]:
def create_filename(prefix=None):
    if prefix is None:
        prefix = 'v'
    d = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    filename = f"{prefix}_{d.replace(' ', '_').replace(':', '-')}"
    return filename


In [26]:
airtable_df.to_csv(
    DATA_FOLDER_PATH / f"db/{create_filename('airtable')}.csv", 
    index=False
)


In [27]:
dropcontact_df = airtable_df[['Prénom', 'Nom', 'Site entreprise', 'Entreprise']]


In [29]:
dropcontact_df = dropcontact_df.rename(columns={'Nom': 'nom', 'Prénom': 'prénom', 'Site entreprise': 'website', 'Entreprise': 'entreprise'})


In [30]:
dropcontact_df = dropcontact_df.sort_values('nom')


In [35]:
dropcontact_df.to_csv(DATA_FOLDER_PATH / f"db/{create_filename(prefix='drop')}.csv", index=False)
